In [ ]:
from synapse.ml.core.platform import find_secret

# Fill in the following lines with your service information
service_name = "<MyOpenAIService>" # Name of your OpenAI service
deployment_name = "<MyModelDepl>" # Name of your deployment in OpenAI
key = find_secret("<MySecret>", "<MyKeyVault>")  # replace this with your secret and keyvault

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

df = spark.read.option('header', 'true') \
                .option('delimiter', '|') \
                .csv('abfss://<container>@<myadls>.dfs.core.windows.net/RestaurantReview.csv')

display(df)

In [ ]:
dfprompt = df.withColumn('prompt',\
                concat(lit('Classify the sentiment of following restaurant review. Classifications: [Positive, Negative , Neutral] \n\nReview: ')\
                , col('Review')\
                ,lit('\n\nClassification:')))\
            

display(dfprompt)

In [ ]:
from synapse.ml.cognitive import OpenAICompletion

completion = (
    OpenAICompletion()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name)
    .setUrl("https://{}.openai.azure.com/".format(service_name))
    #.setMaxTokens(200)
    .setPromptCol("prompt")
    .setErrorCol("error")
    .setOutputCol("response")
)



In [ ]:
from pyspark.sql.functions import col

completed_df = completion.transform(dfprompt).cache()

display(completed_df)


In [ ]:
from pyspark.sql.functions import col

display(
    completed_df.select( 
        col('Restaurant'), 
        col('Review'), 
        col('response.choices.text').getItem(0).alias('openai_sentiment')     
    )
    
)